In [ ]:
%pip install retry
%pip install neo4j pandas

In [ ]:
%pip install pandas neo4j python-dotenv ipykernel

In [3]:
import pandas as pd

In [4]:
product_df = pd.read_csv('../data/Myntra_fashion_products1.csv')
brand_df=pd.read_csv('../data/Brands_id.csv')


In [ ]:
product_df.head()

In [ ]:
brand_df.head()

In [ ]:
product_brand_df = pd.merge(product_df, brand_df, on='s_no')
product_brand_df.head(1)

In [ ]:
product_brand_df.shape

In [ ]:
product_brand_df.columns

In [ ]:
product_brand_df.isna().sum()

In [ ]:
product_brand_df.shape

In [12]:
from dotenv import load_dotenv
import os

In [13]:
%load_ext dotenv
%dotenv

In [14]:
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USER = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

In [ ]:
NEO4J_USER

In [16]:
from neo4j import GraphDatabase

In [17]:
AUTH = (NEO4J_USER, NEO4J_PASSWORD)

In [18]:
with GraphDatabase.driver(NEO4J_URI,auth=AUTH) as driver:
    driver.verify_connectivity()

In [19]:
def insert_data(tx, row):
    tx.run('''
            CREATE (product:Product {
                s_no: $s_no,
                name: $name,
                sku: $sku,
                mpn: $mpn,
                price: $price,
                in_stock: $in_stock,
                currency: $currency,
                images: $images,
                gender: $gender
            })
           MERGE (brand:Brand {
                s_no: $s_no,
                brand_name: $brand_name,
                description: $description
            })
           CREATE (product)-[:MAKER]->(brand)
            ''', row)


In [ ]:
with driver.session() as session:
    for _, row in product_brand_df.iterrows():
        session.write_transaction(insert_data, row.to_dict())